#  Naive Isolation Network Training Script
This script contains the code necessary to train the so called "naive" fault isolation network

# Setup
The script requires tensorflow version 1.15.0

In [1]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 36kB/s 
     |████████████████████████████████| 512kB 47.8MB/s 
     |████████████████████████████████| 3.8MB 60.4MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=f2a0fc73fc38830be720d9c3092ed1d39cef5d1b0b9fbe0f9d83738ae0764bd0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

Package Import and Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import io
import os
import pickle
from pathlib import Path
import random
from collections import Counter
# Authentication for Managing Data
from google.colab import drive
drive.mount('/content/drive')

rootPath = '/content/drive/My Drive/Data/'
name = "isolateNaive2Layer"
stateful = False # boolea to toggle network state memory
debug = False # boolean to toggle debug options
distribute = False # boolean to toggle distribution of training (not functional)
loadModel = False # boolean to toggle loading a saved model

if debug:
    tf.enable_eager_execution()

if loadModel:
    checkpointPath = rootPath + 'Results/Isolation/'
    weightsName = 'weights.07-0.03.hdf5'
startEpoch = 1
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Properly mount the drive by reading a file from the drive

In [ ]:
register = np.zeros(1)
while not np.any(register):
    try:
        with open(rootPath + 'Isolation/Training/FileRegister.csv','r') as f:
            register = np.genfromtxt(f,delimiter = ",")
    except:
        pass
np.shape(register)


(97, 3)

Preprocessing Functions


In [ ]:
def decode_TFRecord(exampleProto):
    # Read TFRecord file
    # Define features
    featureDescription = {
        'x1': tf.VarLenFeature(dtype=tf.float32),
        'y1': tf.VarLenFeature(dtype=tf.float32),
        'z1': tf.VarLenFeature(dtype=tf.float32),
        'vx1': tf.VarLenFeature(dtype=tf.float32),
        'vy1': tf.VarLenFeature(dtype=tf.float32),
        'vz1':tf.VarLenFeature(dtype=tf.float32),
        'x2': tf.VarLenFeature(dtype=tf.float32),
        'y2': tf.VarLenFeature(dtype=tf.float32),
        'z2': tf.VarLenFeature(dtype=tf.float32),
        'vx2': tf.VarLenFeature(dtype=tf.float32),
        'vy2': tf.VarLenFeature(dtype=tf.float32),
        'vz2':tf.VarLenFeature(dtype=tf.float32),
        'x3': tf.VarLenFeature(dtype=tf.float32),
        'y3': tf.VarLenFeature(dtype=tf.float32),
        'z3': tf.VarLenFeature(dtype=tf.float32),
        'vx3': tf.VarLenFeature(dtype=tf.float32),
        'vy3': tf.VarLenFeature(dtype=tf.float32),
        'vz3': tf.VarLenFeature(dtype=tf.float32),
        'x4': tf.VarLenFeature(dtype=tf.float32),
        'y4': tf.VarLenFeature(dtype=tf.float32),
        'z4': tf.VarLenFeature(dtype=tf.float32),
        'vx4': tf.VarLenFeature(dtype=tf.float32),
        'vy4': tf.VarLenFeature(dtype=tf.float32),
        'vz4': tf.VarLenFeature(dtype=tf.float32),
        'label': tf.VarLenFeature(dtype=tf.int64),
        'time': tf.VarLenFeature(dtype=tf.int64),
        'sats': tf.VarLenFeature(dtype=tf.int64)}

    # Extract features from serialized data
    return  tf.io.parse_single_example(exampleProto, featureDescription)



In [ ]:
def preprocess(dataset,seqLen = 50):
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])

    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)
    # take indices only the indices where the time at the beginning of a slice
    # is less than the time at the end of slice (ensures continuity)
    indices = tf.where(time[:-seqLen]<time[seqLen:])
    zeros = tf.zeros_like(indices)
    # the indices need zeros in the second column
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset from the beginning indices
    dsBegin = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels
    dataSlices = dsBegin.map(lambda x: tf.slice(data,x,[seqLen,24]))
    # select labels for data in the same way
    correctLabels =  tf.boolean_mask(labels,time[:-seqLen]<time[seqLen:])
    # Correct labels is a vector with a single 1 (one-hot) in the position
    # of the faulty thruster
    correctLabels = tf.one_hot(correctLabels,36,dtype=tf.int32)
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    dsReturn = tf.data.Dataset.zip((dataSlices,dataLabels))
    return dsReturn

Model Definition Function

In [ ]:
def createIsolateModel(seq_len, batch_size, stateful, 
              num_units):
  source = tf.keras.Input(
  name='seed', shape=(seq_len, 24), 
      batch_size=batch_size)
  
  lstm_1 = tf.keras.layers.LSTM(num_units[0], stateful=stateful, return_sequences=True,dropout=0.0, recurrent_dropout=0.0)(source)
  lstm_2 = tf.keras.layers.LSTM(num_units[1], stateful=stateful, return_sequences=False, dropout=0.0)(lstm_1)
  #lstm_3 = tf.keras.layers.LSTM(num_units[2], stateful=stateful, return_sequences=False, dropout=0.0)(lstm_1)
  dense_1 = tf.keras.layers.Dense(100, activation='relu')(lstm_2)
  
  predict = tf.keras.layers.Dense(36, activation='softmax')(dense_1)
  
  return tf.keras.Model(inputs=[source], outputs=[predict])

# Training Options

In [ ]:
# Load Datasets 
# Parameter Definitions

TFinal = 5602
nSats = 6
# Define model parameters
nUnits = [256,356]
nTimesteps = 50
nEpoch = 10
if debug:
    batchSize = 4096
else:
    batchSize = 4096
learningRate = 0.1


In [ ]:
# Compile Model
if distribute:
    # Create distributed strategy
    # topology = tf.contrib.distribute.initialize_tpu_system()
    #device_assignment = tf.contrib.tpu.DeviceAssignment(topology, core_assignment=tf.contrib.tpu.SINGLE_CORE_ASSIGNMENT)
    #tpu_strategy = tf.contrib.distribute.TPUStrategy(device_assignment=device_assignment)
    #strategy = tf.distribute.experimental.TPUStrategy(resolver)
    strategy = tf.distribute.MirroredStrategy()
    if loadModel:
        detectModel = tf.keras.models.load_model(checkpointPath+weightsName)
    else:
        with strategy.scope():
            detectModel = createIsolateModel(seq_len=nTimesteps, stateful=stateful, num_units=nUnits, batch_size=batchSize)
            adams = tf.keras.optimizers.Adam(learning_rate=learningRate)
            sgd = tf.keras.optimizers.SGD(momentum=0.006)
            detectModel.compile(optimizer=adams,
                            loss=tf.keras.losses.CategoricalCrossentropy(),
                            metrics=['categorical_accuracy']) # Compile with adam optimizer
else:
    if loadModel:
        detectModel = tf.keras.models.load_model(checkpointPath+weightsName)
    else:
        detectModel = createIsolateModel(seq_len=nTimesteps, stateful=stateful, num_units=nUnits, batch_size=batchSize)
    adams = tf.keras.optimizers.Adam(learning_rate=learningRate)
    detectModel.compile(optimizer=adams,
                        loss=tf.keras.losses.CategoricalCrossentropy(),
                        metrics=['categorical_accuracy']) # Compile with adam optimizer
tf.keras.utils.plot_model(
    detectModel, to_file=rootPath + 'naiveIsolateModel.png', show_shapes=True, show_layer_names=True,
    rankdir='LR', expand_nested=False, dpi=96)

In [ ]:
# Load Datasets
pathTrain = rootPath + 'Isolation/Training/'
listdirTrain = []
if debug:
    globTrain = pathTrain + 'TrainCorrected_[0].tfrecord'
    listdirTrain = tf.io.gfile.glob(globTrain)
else:
    globTrain = pathTrain + 'TrainCorrected_[0-5].tfrecord'
    listdirTrain = tf.io.gfile.glob(globTrain)
globTrain = pathTrain + 'TrainCorrected_[0-9].tfrecord'
# Create File List
listdirTrain = tf.io.gfile.glob(globTrain)
nFilesTrain = len(listdirTrain)
# Validation set
pathTest = rootPath + 'Isolation/Testing/'
if debug:
    globTest = pathTest + 'TestCorrected_0.tfrecord'
else:
    globTest = pathTest + 'TestCorrected_[0].tfrecord'
listdirTest = tf.io.gfile.glob(globTest)

['/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_0.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_1.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_2.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_3.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_4.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_5.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_6.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_7.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_8.tfrecord', '/content/drive/My Drive/Data/Isolation/Training/TrainCorrected_9.tfrecord']


In [ ]:
# Batch, shuffle and repeat dataset
fileListDatasetTrain = tf.data.TFRecordDataset(listdirTrain)
decodedDataset = fileListDatasetTrain.map(decode_TFRecord).cache()
processedDataset = decodedDataset.flat_map(preprocess)
trainDataset = processedDataset
if not stateful:
    trainDataset = trainDataset.shuffle(10*batchSize)
trainDataset = trainDataset.repeat(nEpoch).batch(batchSize,drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)

# Validation Dataset
nFilesTest = len(listdirTest)
fileListDatasetTest = tf.data.TFRecordDataset(listdirTest).cache()
decodeValDataset = fileListDatasetTest.map(decode_TFRecord)
processedVal = decodeValDataset.flat_map(preprocess)
validationDataset = processedVal.batch(batchSize,drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)
# Determine steps per epoch
trainSteps = int(nFilesTrain*6*100*(TFinal-nTimesteps)/batchSize)
testSteps = int(nFilesTest*6*100*(TFinal-nTimesteps)/batchSize)

print("Training Files: {}".format(nFilesTrain))
print("Testing Files: {}".format(nFilesTest))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
['/content/drive/My Drive/Data/Isolation/Testing/TestCorrected_0.tfrecord']
Training Files: 10
Testing Files: 1


# Training

In [ ]:
# Checkpoint to save the model every two epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(rootPath + "Results/Isolation/"+name+"_weights.{epoch:02d}-{categorical_accuracy:.2f}.hdf5", 
                                                monitor='categorical_accuracy', verbose=0, save_best_only=False, 
                                                save_weights_only=False, mode='auto', save_freq = 4879)
# Stopper to stop training if loss does not improve three times in a row
stopper = tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss')
VAL = True
history = detectModel.fit(trainDataset, 
                             epochs=nEpoch, steps_per_epoch = trainSteps,
                            callbacks=[checkpoint,stopper], 
                            initial_epoch = startEpoch-1,
                            validation_data= validationDataset if VAL else None,
                            validation_steps = testSteps if VAL else None)

In [ ]:
# Saving to local and to google drive
detectModel.save(rootPath +'Results/Isolation/{0}.hdf5'.format(name), overwrite=True)
detectModel.save_weights(rootPath + 'Results/Isolation/weights_{0}.h5'.format(name), overwrite=True)

# Saving the training history
with open(rootPath + 'Results/Isolation/trainHistoryDict.pkl', 'wb') as file_pi:
  pickle.dump(history.history, file_pi)